In [21]:
# Load the environment variables
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("env_vars.env")) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Hugging Face Hub LLM

In [22]:
from langchain import PromptTemplate
from langchain import HuggingFaceHub, LLMChain

template = """Question: {question} Answer: """
prompt = PromptTemplate(template=template, input_variables=['question'])
print(prompt)

# Initialize Hub LLM
hub_llm = HuggingFaceHub(repo_id='google/flan-t5-xl', model_kwargs={'temperature':1e-10})

# Create prompt template > LLM chain
llm_chain = LLMChain(prompt=prompt, llm=hub_llm)

input_variables=['question'] output_parser=None partial_variables={} template='Question: {question} Answer: ' template_format='f-string' validate_template=True


In [26]:
# Ask the question
question = "Who is the current chairman of the Federal Reserve?"
print(question)

# NOTE: This will likely time out!
# print(llm_chain.run(question))

Who is the current chairman of the Federal Reserve?


# OpenAI LLM

In [25]:
from langchain.llms import OpenAI

# Initialize the model
davinci = OpenAI(model_name="text-davinci-003")
llm_chain = LLMChain(prompt=prompt, llm=davinci)

In [27]:
# Ask the question
print(llm_chain.run(question))



The current chairman of the Federal Reserve is Jerome Powell.


# Prompt Templates

Two kinds of knowledge that can be "reasoned" by LLM:
- **Parametric knowlege** -- learned during training and stored in model parameters (weights)
- **Source knowledge** -- provided to the model at inference time as examples (via input prompt)

In [29]:
prompt = """The following is a conversation with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: User: What is the meaning of life? AI: """

openai.temperature = 1.0 # increase creativity/randomness of output
print(davinci(prompt))



The meaning of life is to find your own purpose and make the most of it. Life is what you make it!


In [31]:
# To help the model, we can give it a few examples of the type of answers we’d like
prompt = """The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: User: How are you? AI: I can't complain but sometimes I still do. 
User: What time is it? AI: It's time to get a watch. User: What is the meaning of life? AI: """

print(davinci(prompt))


The meaning of life is to live it to the fullest and enjoy every moment.


In [32]:
# FewShotPromptTemplate caters to source knowledge input
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, 
    {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """ User: {query} AI: {answer} """

In [34]:
# create a prompt example from above template
example_prompt = PromptTemplate(input_variables=["query", "answer"], template=example_template)
print(example_prompt)

input_variables=['query', 'answer'] output_parser=None partial_variables={} template=' User: {query} AI: {answer} ' template_format='f-string' validate_template=True


In [35]:
# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: """

# and the suffix our user input and output indicator
suffix = """ User: {query} AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(examples=examples, example_prompt=example_prompt,
                                                 prefix=prefix, suffix=suffix, input_variables=["query"],
                                                 example_separator="\n\n")
print(few_shot_prompt_template)

input_variables=['query'] output_parser=None partial_variables={} examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}] example_selector=None example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True) suffix=' User: {query} AI: ' example_separator='\n\n' prefix='The following are exerpts from conversations with an AI assistant. \nThe assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. \nHere are some examples: ' template_format='f-string' validate_template=True


In [36]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: 

 User: How are you? AI: I can't complain but sometimes I still do. 

 User: What time is it? AI: It's time to get a watch. 

 User: What is the meaning of life? AI: 


In [37]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [38]:
print(examples)

[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}]


In [39]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(examples=examples,
                                              example_prompt=example_prompt,
                                              max_length=50 # this sets the max length that examples should be
                                             )
print(example_selector)

examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}] example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True) get_text_length=<function _get_length_based at 0x11b59e3b0> max_length=50 example_text_lengths=[15, 14, 11, 16, 10, 19, 19]


In [40]:
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector, # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)
print(dynamic_prompt_template)

input_variables=['query'] output_parser=None partial_variables={} examples=None example_selector=LengthBasedExampleSelector(examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}], example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True), get_text_length=<function _get_length_based at 0x11b59e3b0>, max_length=50,

In [43]:
prompt = dynamic_prompt_template.format(query="How do birds fly?")
print(prompt)

The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: 
 User: How are you? AI: I can't complain but sometimes I still do. 
 User: What time is it? AI: It's time to get a watch. 
 User: What is the meaning of life? AI: 42 
 User: How do birds fly? AI: 


In [44]:
print(davinci(prompt))

 With a little help from their friends!


# Memory

In [47]:
from langchain import OpenAI
from langchain.chains import ConversationChain

# first initialize the large language model
llm = OpenAI(temperature=0, model_name="text-davinci-003")

# now initialize the conversation chain
conversation = ConversationChain(llm=llm)
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


## ConversationBufferMemory

In [48]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(llm=llm, memory=ConversationBufferMemory())
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': " Good morning! It's a beautiful day today, isn't it? How can I help you?"}

In [49]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [50]:
count_tokens(conversation_buf,
             "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
            )

Spent a total of 178 tokens


' Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?'

In [51]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI:  Good morning! It's a beautiful day today, isn't it? How can I help you?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI:  Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?


## ConversationSummaryMemory

In [52]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(llm=llm, memory=ConversationSummaryMemory(llm=llm))
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [53]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(conversation_sum, "Good morning AI!")

Spent a total of 283 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [55]:
count_tokens(conversation_sum, 
             "My interest here is to get filthy rich and live a lavish life!"
            )

Spent a total of 383 tokens


' That sounds like an exciting goal! What kind of steps do you think you need to take to achieve it?'

In [56]:
count_tokens(conversation_sum, 
             "I just want to analyze the different possibilities to that goal. What can you think of?"
            )

Spent a total of 539 tokens


" That's a great goal! I think the first step is to identify what resources you have available to you. Do you have any savings, investments, or other assets that you can use to help you reach your goal?"

In [57]:
print(conversation_sum.memory.buffer)


The human greeted the AI and the AI responded with a greeting and asked how it could help. The human expressed their goal of getting rich and living a lavish life, to which the AI responded positively and asked what steps the human thought they needed to take to achieve it. The human asked the AI to analyze the different possibilities to that goal, and the AI suggested the first step should be to identify what resources the human had available to them.


## ConversationBufferWindowMemory

In [63]:
# Acts in the same way as our earlier “buffer memory” but adds a window to the memory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# We set k=1 -- meaning the window will remember the single latest interaction between the human and AI
conversation_bufw = ConversationChain(llm=llm, memory=ConversationBufferWindowMemory(k=1))

In [64]:
bufw_history = conversation_bufw.memory.load_memory_variables(inputs=[])['history']
print(bufw_history)

## ConversationSummaryBufferMemory

In [67]:
# Summarizes the earliest interactions in a conversation while maintaining the most recent tokens in the conversation
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

conversation_sum_bufw = ConversationChain(llm=llm, 
                                          memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=650)
                                         )

This the only one of our memory types (so far) that allows us to remember distant interactions and store the most recent interactions in their raw form.

In [68]:
# Others:
# Knowledge graph
from langchain.chains.conversation.memory import ConversationKGMemory

# Entities -- remember details about the entities (persons, etc.)
from langchain.chains.conversation.memory import ConversationEntityMemory

# Hallucinations